In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, StandardScaler

import tensorflow as tf
print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [2]:
def scale_data(df):

    scaled_features = StandardScaler().fit_transform(df.values)
    df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)
 
    return(df)

In [3]:
df = pd.read_csv('data/rolled_df.csv')
print(len(df))
#df = df.drop(['month', 'day', 'hour', 'min'], axis=1)
df.head(10)

24821


,month,day,hour,min,0,1,2,3,4,5,6,7,8,9,10,11,12,13,direction
0,1,1,20,47,0.00009,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,decrease
1,1,1,21,2,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,increase
2,1,1,21,17,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,same
3,1,1,21,32,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,decrease
4,1,1,21,47,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,decrease
5,1,1,22,2,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,increase
6,1,1,22,17,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,same
7,1,1,22,32,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,increase
8,1,1,22,47,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,decrease
9,1,1,23,2,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,-0.00021,decrease


In [4]:
mapping = {'decrease': 0, 'same': 1, 'increase': 2}
df = df.replace({'direction': mapping})

In [5]:
target_col = 'direction'

In [6]:
train_x, valid_x, train_y, valid_y = train_test_split(df.loc[:, df.columns != target_col], df[target_col],train_size = 0.8,random_state = 1)

In [7]:
#train_x = scale_data(train_x)
#valid_x = scale_data(valid_x)
#train_x.head()

In [8]:
Layer_nodes = 512

In [9]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(Layer_nodes, activation='relu', input_shape=[len(train_x.keys())]),
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),              
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(Layer_nodes, activation='relu'),      
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [10]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',    
    metrics=['accuracy'])

In [11]:
epoch_val = 3

In [12]:
model.fit(train_x, train_y, epochs=epoch_val)

Epoch 1/3
621/621 [==============================] - 2s 3ms/step - loss: 52.9042 - accuracy: 0.3636
Epoch 2/3
621/621 [==============================] - 2s 3ms/step - loss: 261.2412 - accuracy: 0.3637
Epoch 3/3
621/621 [==============================] - 2s 3ms/step - loss: 369.8862 - accuracy: 0.3637


In [13]:
test_loss, test_accuracy = model.evaluate(valid_x, valid_y)
print(f'Test Loss {test_loss}')
print(f'Test Accuracy {test_accuracy}')

156/156 [==============================] - 0s 2ms/step - loss: -472.5865 - accuracy: 0.3605
Test Loss -472.5864562988281
Test Accuracy 0.36052367091178894


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier()
clf.fit(train_x, train_y)

target_names = ['decrease', 'same', 'increase']
print(classification_report(valid_y, clf.predict(valid_x),target_names=target_names))